In [1]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
import numpy as np
import os
from model import detr_simplified, DETR, DETRModel
from cornell_dataset import CornellDataset
from view_bbox import draw_bbox

In [2]:
model = DETRModel(num_classes=20, num_queries=8)
loaded_model = torch.load("overfit-third/model_299.ckpt")

model.load_state_dict(loaded_model["model"])
model.eval()
m = 0

Using cache found in /home/sbhara/.cache/torch/hub/facebookresearch_detr_main


In [3]:
from torch.utils.data import DataLoader 
dataset_path = "dataset/cornell"
img_set = "train"
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])
inv_normalize = transforms.Normalize(
                            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
                            std=[1/0.229, 1/0.224, 1/0.225])
train_dataset = CornellDataset(dataset_path, "train", normalize)
val_dataset = CornellDataset(dataset_path, "val", normalize)

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox).to("cpu")
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [4]:
def iou(box1,box2):
  '''
    Computes the intersection over union of two bounding boxes box = [x1,y1,x2,y2]
    where (x1, y1) and (x2, y2) are the top left and bottom right coordinates respectively
  '''
  x1, y1 = max(box1[0], box2[0]), max(box1[1], box2[1])
  x2, y2 = min(box1[2], box2[2]), min(box1[3], box2[3])
  inter_area = max(0, (x2 - x1 + 1)) * max(0, (y2 - y1 + 1))
  union_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1) + (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1) - inter_area
  return inter_area/union_area

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
final_iou = 0
correct_pred = 0
for i, (x_val, y_val) in enumerate(train_loader):
    x_val = x_val.to(device)
    box = y_val
    output = model(x_val)
    bbox = output["pred_boxes"]
    labels = output["pred_logits"]

    probas = labels.softmax(-1)
    #print( bbox * 224)
    probas = torch.argmax(probas, dim=2, keepdim=True)
    idx = torch.argmax(probas)

    pred_class = probas[:, idx].squeeze().cpu().numpy()
    bbox_pred = bbox[:, idx].detach().cpu() * 224

    x_val = inv_normalize(x_val)

    draw_func = draw_bbox(bbox_pred.squeeze(0).detach().cpu().numpy(), pred_class)
    draw_func.draw_rotated_box(x_val[0].permute(1, 2, 0).cpu().numpy(), fig_num=i)
    iou_calc = iou(y_val[1].squeeze(0).detach().cpu().numpy()*224, bbox_pred.squeeze(0).detach().cpu().numpy())
    final_iou += iou_calc
    if pred_class == y_val[0].numpy():
        correct_pred += 1

    print(f"IOU: {iou_calc}, GT: {y_val[0]}, {y_val[1]*224} Class: {pred_class}, bboxes: {bbox_pred}")
    i+=1
    if i == 10:
        print(f"Mean IoU: {final_iou/i * 100}, accuracy: {correct_pred/i * 100}")
        break
    

/home/sbhara/.cache/torch/hub/facebookresearch_detr_main/models/position_encoding.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  dim_t = self.temperature ** (2 * (dim_t // 2) / self.num_pos_feats)


TypeError: draw_rotated_box() got an unexpected keyword argument 'fig'